In [1]:
# Cell 1: imports & activation helpers

import math
import random

def sigmoid(x: float) -> float:
    """Sigmoid activation function."""
    return 1.0 / (1.0 + math.exp(-x))

def dsigmoid_from_output(y: float) -> float:
    """
    Derivative of sigmoid, expressed in terms of its output y.
    If y = sigmoid(x), then d/dx sigmoid(x) = y * (1 - y).
    """
    return y * (1.0 - y)


In [2]:
# Cell 2: simple feed-forward NN for Morse → letter

class SimpleMorseNN:
    """
    Simple 1-hidden-layer neural network with sigmoid activations,
    implemented in plain Python.

    Architecture:
    - input layer: 3 * max_morse_len (one-hot for dot/dash/void per time step)
    - hidden layer: configurable size
    - output layer: number of classes (letters A–E) with one-hot targets
    """
    def __init__(self, input_size, hidden_size, output_size, lr=0.5, seed=0):
        self.lr = lr
        random.seed(seed)

        # weights: W1[j][i] connects input i -> hidden j
        self.W1 = [
            [(random.random() - 0.5) * 0.2 for _ in range(input_size)]
            for _ in range(hidden_size)
        ]
        self.b1 = [(random.random() - 0.5) * 0.2 for _ in range(hidden_size)]

        # weights: W2[k][j] connects hidden j -> output k
        self.W2 = [
            [(random.random() - 0.5) * 0.2 for _ in range(hidden_size)]
            for _ in range(output_size)
        ]
        self.b2 = [(random.random() - 0.5) * 0.2 for _ in range(output_size)]

        # storage for forward pass
        self.last_input = None
        self.last_h1 = None
        self.last_y = None
        self.last_z1 = None
        self.last_z2 = None

    def forward(self, x):
        """
        Forward pass.
        x: input vector (list or tuple of length input_size).
        Returns: output activations (list of length output_size).
        """
        # hidden layer
        z1 = []
        h1 = []
        for j in range(len(self.W1)):
            s = sum(w * x_i for w, x_i in zip(self.W1[j], x)) + self.b1[j]
            z1.append(s)
            h1.append(sigmoid(s))

        # output layer
        z2 = []
        y = []
        for k in range(len(self.W2)):
            s = sum(w * h for w, h in zip(self.W2[k], h1)) + self.b2[k]
            z2.append(s)
            y.append(sigmoid(s))

        # store for backprop
        self.last_input = x
        self.last_h1 = h1
        self.last_y = y
        self.last_z1 = z1
        self.last_z2 = z2

        return y

    def backward(self, target):
        """
        Backward pass (one step of backprop + weight update)
        using squared error: E = 0.5 * sum_k (y_k - d_k)^2

        target: list of desired outputs (one-hot).
        Returns: scalar error E for this pattern.
        """
        x = self.last_input
        h1 = self.last_h1
        y = self.last_y

        # --- deltas for output layer ---
        delta2 = []
        for k in range(len(y)):
            error = y[k] - target[k]                 # ∂E/∂y_k
            delta = error * dsigmoid_from_output(y[k])  # δ_k^(2)
            delta2.append(delta)

        # --- deltas for hidden layer ---
        delta1 = []
        for j in range(len(h1)):
            downstream = sum(delta2[k] * self.W2[k][j] for k in range(len(delta2)))
            delta = downstream * dsigmoid_from_output(h1[j])  # δ_j^(1)
            delta1.append(delta)

        # --- update output weights and biases ---
        for k in range(len(self.W2)):
            for j in range(len(self.W2[k])):
                self.W2[k][j] -= self.lr * delta2[k] * h1[j]
            self.b2[k] -= self.lr * delta2[k]

        # --- update hidden weights and biases ---
        for j in range(len(self.W1)):
            for i in range(len(self.W1[j])):
                self.W1[j][i] -= self.lr * delta1[j] * x[i]
            self.b1[j] -= self.lr * delta1[j]

        # --- compute error E = 0.5 * sum (y - d)^2 ---
        E = 0.5 * sum((y[k] - target[k]) ** 2 for k in range(len(y)))
        return E


In [3]:
# Cell 3: Morse symbol encoding & training data

# one-hot per symbol:
# dot '.', dash '-', void '_' (padding)
symbol_vectors = {
    '.': [1, 0, 0],  # dot
    '-': [0, 1, 0],  # dash
    '_': [0, 0, 1],  # void / padding
}

def encode_morse(seq: str, max_len: int):
    """
    Encode a Morse string into a flat vector of length max_len * 3.
    seq: string like ".-" (no padding symbols inside this string)
    max_len: maximum Morse length over all symbols (here: 4)
    """
    vec = []
    # actual symbols
    for c in seq:
        vec.extend(symbol_vectors[c])
    # pad with void symbols
    for _ in range(max_len - len(seq)):
        vec.extend(symbol_vectors['_'])
    return vec

# Morse definitions for letters A–E
morse_dict = {
    'A': '.-',
    'B': '-...',
    'C': '-.-.',
    'D': '-..',
    'E': '.',
}

# determine max length from the codes we use
max_len = max(len(code) for code in morse_dict.values())
input_size = max_len * 3

# alphabet / class list
letters = list(morse_dict.keys())

# build training data
X_train = [encode_morse(morse_dict[ch], max_len) for ch in letters]

Y_train = []
for ch in letters:
    vec = [0] * len(letters)
    vec[letters.index(ch)] = 1  # one-hot target
    Y_train.append(vec)

print("Letters:", letters)
print("Max Morse length:", max_len)
print("Input vector size:", input_size)
print("Example: 'A' encoded:", X_train[0])
print("Example: target for 'A':", Y_train[0])


Letters: ['A', 'B', 'C', 'D', 'E']
Max Morse length: 4
Input vector size: 12
Example: 'A' encoded: [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
Example: target for 'A': [1, 0, 0, 0, 0]


In [4]:
# Cell 4: training loop

hidden_size = 8       # number of hidden neurons
output_size = len(letters)
learning_rate = 0.5
epochs = 2000

nn = SimpleMorseNN(input_size, hidden_size, output_size,
                   lr=learning_rate, seed=0)

errors_per_epoch = []

for epoch in range(epochs):
    total_error = 0.0
    # one epoch = loop over all training patterns once
    for x, y in zip(X_train, Y_train):
        nn.forward(x)                # forward pass
        total_error += nn.backward(y)  # backward pass + weight update
    errors_per_epoch.append(total_error)

    if epoch % 200 == 0:
        print(f"Epoch {epoch:4d} | total error = {total_error:.6f}")

print("Training finished.")


Epoch    0 | total error = 2.907471
Epoch  200 | total error = 0.495248
Epoch  400 | total error = 0.051464
Epoch  600 | total error = 0.023627
Epoch  800 | total error = 0.015004
Epoch 1000 | total error = 0.010884
Epoch 1200 | total error = 0.008494
Epoch 1400 | total error = 0.006941
Epoch 1600 | total error = 0.005855
Epoch 1800 | total error = 0.005055
Training finished.


In [5]:
# Cell 5: test predictions on the training letters (A–E)

def predict_letter(morse_seq: str):
    """
    Feed a Morse sequence to the trained network and return:
    - predicted letter
    - raw output activations
    """
    x = encode_morse(morse_seq, max_len)
    y = nn.forward(x)
    # argmax over outputs
    k = max(range(len(y)), key=lambda i: y[i])
    return letters[k], y

for ch, seq in morse_dict.items():
    pred, y = predict_letter(seq)
    y_str = [f"{v:.3f}" for v in y]
    print(f"Morse {seq:4s}  | target {ch}  | predicted {pred}  | outputs {y_str}")


Morse .-    | target A  | predicted A  | outputs ['0.972', '0.002', '0.001', '0.012', '0.018']
Morse -...  | target B  | predicted B  | outputs ['0.016', '0.972', '0.024', '0.022', '0.000']
Morse -.-.  | target C  | predicted C  | outputs ['0.006', '0.020', '0.972', '0.000', '0.016']
Morse -..   | target D  | predicted D  | outputs ['0.012', '0.024', '0.001', '0.969', '0.013']
Morse .     | target E  | predicted E  | outputs ['0.024', '0.000', '0.022', '0.021', '0.973']


In [6]:
# Cell 6 (optional): some experiments

# Example: feed a padded version of 'A' that is already padded (no change)
pred, y = predict_letter(".-")
print("\nExperiment 1: A (.-)  → predicted:", pred)

# Example: feed B with one missing dot (should likely misclassify)
try_seq = '-..'  # true B is '-...'
pred, y = predict_letter(try_seq)
print("Experiment 2: '-..' (truncated B) → predicted:", pred, "outputs:", [f"{v:.3f}" for v in y])



Experiment 1: A (.-)  → predicted: A
Experiment 2: '-..' (truncated B) → predicted: D outputs: ['0.012', '0.024', '0.001', '0.969', '0.013']


In [7]:
import math
import random

# ----- activation and derivative (sigmoid) -----

def sigmoid(x: float) -> float:
    """Sigmoid activation: squashes any real value into (0,1)."""
    return 1.0 / (1.0 + math.exp(-x))

def dsigmoid_from_output(y: float) -> float:
    """
    Derivative of sigmoid using its output.
    If y = sigmoid(x), then derivative w.r.t x is y * (1 - y).
    """
    return y * (1.0 - y)


# ----- neural network with backpropagation -----

class SimpleMorseNN:
    def __init__(self, input_size: int, hidden_size: int, output_size: int,
                 lr: float = 0.5, seed: int = 0):
        self.lr = lr
        random.seed(seed)

        # weights input -> hidden: W1[j][i]
        self.W1 = [
            [(random.random() - 0.5) * 0.2 for _ in range(input_size)]
            for _ in range(hidden_size)
        ]
        self.b1 = [(random.random() - 0.5) * 0.2 for _ in range(hidden_size)]

        # weights hidden -> output: W2[k][j]
        self.W2 = [
            [(random.random() - 0.5) * 0.2 for _ in range(hidden_size)]
            for _ in range(output_size)
        ]
        self.b2 = [(random.random() - 0.5) * 0.2 for _ in range(output_size)]

        # placeholders to remember last forward pass
        self.last_input = None
        self.last_hidden = None
        self.last_output = None

    def forward(self, x):
        """
        Forward pass: compute hidden activations and outputs for input x.
        Stores values so backward() can reuse them.
        """
        # hidden layer
        h = []
        for j in range(len(self.W1)):
            s = sum(w * x_i for w, x_i in zip(self.W1[j], x)) + self.b1[j]
            h.append(sigmoid(s))

        # output layer
        y = []
        for k in range(len(self.W2)):
            s = sum(w * h_j for w, h_j in zip(self.W2[k], h)) + self.b2[k]
            y.append(sigmoid(s))

        # store for backprop
        self.last_input = x
        self.last_hidden = h
        self.last_output = y

        return y

    def backward(self, target):
        """
        Backward pass (backpropagation):
        - uses last forward pass
        - updates weights and biases
        - returns error for this sample
        """
        x = self.last_input
        h = self.last_hidden
        y = self.last_output

        # ---- 1) Output layer error signals ----
        delta_out = []
        for k in range(len(y)):
            # difference between actual and desired
            error_k = y[k] - target[k]
            # scale by how sensitive sigmoid is at this point
            delta_k = error_k * dsigmoid_from_output(y[k])
            delta_out.append(delta_k)

        # ---- 2) Hidden layer error signals ----
        delta_hid = []
        for j in range(len(h)):
            # sum of contributions from all output neurons that depend on this hidden neuron
            downstream = sum(delta_out[k] * self.W2[k][j] for k in range(len(delta_out)))
            delta_j = downstream * dsigmoid_from_output(h[j])
            delta_hid.append(delta_j)

        # ---- 3) Update output weights and biases ----
        for k in range(len(self.W2)):
            for j in range(len(self.W2[k])):
                # move weight in direction that reduces error
                self.W2[k][j] -= self.lr * delta_out[k] * h[j]
            # update bias (acts like a weight with constant input 1)
            self.b2[k] -= self.lr * delta_out[k]

        # ---- 4) Update hidden weights and biases ----
        for j in range(len(self.W1)):
            for i in range(len(self.W1[j])):
                self.W1[j][i] -= self.lr * delta_hid[j] * x[i]
            self.b1[j] -= self.lr * delta_hid[j]

        # ---- 5) Compute overall error for this sample ----
        E = 0.5 * sum((y[k] - target[k])**2 for k in range(len(y)))
        return E


In [8]:
nn = SimpleMorseNN(input_size=12, hidden_size=8, output_size=5, lr=0.5)

x_dummy = [0.0] * 12
y_dummy = nn.forward(x_dummy)
E_dummy = nn.backward([1, 0, 0, 0, 0])

print("Output:", y_dummy)
print("Error:", E_dummy)


Output: [0.46585223785796287, 0.492556174674926, 0.49870074803411124, 0.4889622334122144, 0.4964410618473407]
Error: 0.6310828233465503


In [9]:
# --- Symbol encoding (dot, dash, void) ---

symbol_vectors = {
    '.': [1, 0, 0],  # dot
    '-': [0, 1, 0],  # dash
    '_': [0, 0, 1],  # void / padding
}

def encode_morse(seq: str, max_len: int):
    """
    Encode a Morse string into a flat vector of length max_len * 3.
    Pads with '_' (void) on the right.
    """
    vec = []
    # actual symbols
    for c in seq:
        vec.extend(symbol_vectors[c])
    # pad to max_len
    for _ in range(max_len - len(seq)):
        vec.extend(symbol_vectors['_'])
    return vec

# Morse codes for A–E
morse_dict = {
    'A': '.-',
    'B': '-...',
    'C': '-.-.',
    'D': '-..',
    'E': '.',
}

# figure out max length and input size
max_len = max(len(code) for code in morse_dict.values())
input_size = max_len * 3

# list of classes in order
letters = list(morse_dict.keys())

# build X (inputs) and Y (one-hot targets)
X_train = [encode_morse(morse_dict[ch], max_len) for ch in letters]

Y_train = []
for ch in letters:
    target = [0] * len(letters)
    target[letters.index(ch)] = 1  # one-hot
    Y_train.append(target)

print("Letters:", letters)
print("Max Morse length:", max_len)
print("Input size:", input_size)
print("Example 'A' encoded:", X_train[0])
print("Target for 'A':", Y_train[0])


Letters: ['A', 'B', 'C', 'D', 'E']
Max Morse length: 4
Input size: 12
Example 'A' encoded: [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
Target for 'A': [1, 0, 0, 0, 0]


In [10]:
# --- Training the network ---

hidden_size = 8
output_size = len(letters)
learning_rate = 0.5
epochs = 2000

nn = SimpleMorseNN(input_size, hidden_size, output_size,
                   lr=learning_rate, seed=0)

errors_per_epoch = []

for epoch in range(epochs):
    total_error = 0.0
    for x, y in zip(X_train, Y_train):
        nn.forward(x)
        total_error += nn.backward(y)
    errors_per_epoch.append(total_error)

    if epoch % 200 == 0:
        print(f"Epoch {epoch:4d} | total error = {total_error:.6f}")

print("Training finished.")


Epoch    0 | total error = 2.907471
Epoch  200 | total error = 0.495248
Epoch  400 | total error = 0.051464
Epoch  600 | total error = 0.023627
Epoch  800 | total error = 0.015004
Epoch 1000 | total error = 0.010884
Epoch 1200 | total error = 0.008494
Epoch 1400 | total error = 0.006941
Epoch 1600 | total error = 0.005855
Epoch 1800 | total error = 0.005055
Training finished.


In [ ]:
# --- Testing: predict letters from Morse sequences ---

def predict_letter(morse_seq: str):
    x = encode_morse(morse_seq, max_len)
    y = nn.forward(x)
    # argmax over outputs
    k = max(range(len(y)), key=lambda i: y[i])
    return letters[k], y

for ch, seq in morse_dict.items():
    pred, y = predict_letter(seq)
    y_str = [f"{v:.3f}" for v in y]
    print(f"Morse {seq:4s} | target {ch} | predicted {pred} | outputs {y_str}")
